
# Informations sur l'étudiant:

* Nom: ANDRIANARAHINJAKA
* Prénom(s): Yohannee Aintsoa
* Classe: IGGLIA
* Numéro: .

# Analyse de données avec Régression Linéaire

## Jeu de données: California Housing (prix des logements en Californie)

Ce notebook présente un workflow complet d'analyse de données en utilisant un modèle de **régression linéaire** pour prédire le **prix médian des logements** en Californie à partir de caractéristiques géographiques et démographiques.

##1. Chargement et inspection des données


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Configuration graphique
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("Bibliothèques importées avec succès")

Matplotlib is building the font cache; this may take a moment.


Bibliothèques importées avec succès
